In [1]:
import pandas as pd
import numpy as np

In [3]:
pricing_df = pd.read_csv("final_outputs/pricing_signals.csv")
competitor_df = pd.read_csv("final_outputs/competitor_signals.csv")
sales_df = pd.read_csv("final_outputs/sales_signals.csv")
inventory_df = pd.read_csv("final_outputs/inventory_signals.csv")
ads_df = pd.read_csv("final_outputs/ads_signals.csv")
returns_df = pd.read_csv("final_outputs/returns_signals.csv")


In [4]:
final_df = pricing_df.copy()

final_df = final_df.merge(competitor_df, on="SKU", how="left")
final_df = final_df.merge(sales_df, on="SKU", how="left")
final_df = final_df.merge(inventory_df, on="SKU", how="left")
final_df = final_df.merge(ads_df, on="SKU", how="left")
final_df = final_df.merge(returns_df, on="SKU", how="left")


In [5]:
final_df["Ads_Pricing_Flag"] = final_df["Ads_Pricing_Flag"].fillna("Neutral")
final_df["High_Return_Risk"] = final_df["High_Return_Risk"].fillna(False)


In [6]:
final_df["Final_Pricing_Recommendation"] = "Hold Price"

In [7]:
final_df.loc[
    (final_df["Inventory_Pricing_Signal"] == "Reduce Price") &
    (final_df["Ads_Pricing_Flag"] != "Visibility Issue") &
    (final_df["High_Return_Risk"] == False),
    "Final_Pricing_Recommendation"
] = "Reduce Price"


In [9]:
final_df.columns

Index(['SKU', 'Current_Price', 'Total_Cost', 'Floor_Price', 'Target_Price',
       'Below_Floor_Price', 'Competition_Level', 'Pricing_Power_Index',
       'Market_Type', 'High_Competitive_Pressure', 'Sessions - Total',
       'Units Ordered', 'Conversion_Rate', 'Revenue_per_Session',
       'Traffic_Level', 'Demand_Level', 'Pricing_Action',
       'Low_Conversion_Flag', 'High_Demand_Flag', 'Demand_CV',
       'Total_Inventory', 'Net_Sellable_Inventory', 'days-of-supply',
       'Inventory_Status', 'Sell_Through_Level', 'Inventory_Pricing_Signal',
       'Inventory_Risk_Flag', 'CTR', 'ACOS', 'Ads_Pricing_Flag', 'returns_7d',
       'returns_30d', 'returns_90d', 'Return_Trend', 'High_Return_Risk',
       'Final_Pricing_Recommendation'],
      dtype='object')

In [10]:
final_df = final_df.rename(columns={
    "Pricing_Action": "Sales_Pricing_Action"
})

In [11]:
final_df.loc[
    (final_df["Inventory_Pricing_Signal"] == "Increase Price") &
    (final_df["Sales_Pricing_Action"] == "Increase Price") &
    (final_df["High_Return_Risk"] == False),
    "Final_Pricing_Recommendation"
] = "Increase Price"


In [12]:
#Assign final recommended price
final_df["Recommended_Price"] = final_df["Current_Price"]

final_df.loc[
    final_df["Final_Pricing_Recommendation"] == "Reduce Price",
    "Recommended_Price"
] = final_df["Floor_Price"]

final_df.loc[
    final_df["Final_Pricing_Recommendation"] == "Increase Price",
    "Recommended_Price"
] = final_df["Target_Price"]

In [13]:
final_pricing_table = final_df[[
    "SKU",
    "Current_Price",
    "Floor_Price",
    "Target_Price",
    "Competition_Level",
    "Demand_Level",
    "Inventory_Status",
    "Sell_Through_Level",
    "Ads_Pricing_Flag",
    "High_Return_Risk",
    "Final_Pricing_Recommendation",
    "Recommended_Price"
]]

In [14]:
final_pricing_table.to_csv(
    "final_outputs/final_pricing_recommendation.csv",
    index=False
)

"This project designs a realistic, data-driven pricing strategy by converting multiple business datasets into actionable pricing signals. Cost data defines safe pricing boundaries, while sales, inventory, and competitor data guide when prices should be adjusted. Advertising and returns data act as validation layers to ensure pricing actions are applied only where visibility and product quality are not limiting factors. The resulting framework produces SKU-level recommended prices using a structured decision logic that balances profitability, demand response, inventory risk, and market pressure. This approach demonstrates how pricing decisions can be systematically generated and safely deployed in a real business environment."

In [16]:
### framework for adjusting prices
"""
IF 
    inventory + demand justify change AND ads show sufficient visibility 
    AND returns risk is low AND price stays within floor/target bounds
→ Adjust price
ELSE
→ Hold price
"""

'\nIF \n    inventory + demand justify change AND ads show sufficient visibility \n    AND returns risk is low AND price stays within floor/target bounds\n→ Adjust price\nELSE\n→ Hold price\n'